# Übung 9 - Einführung Assoziationsanalyse

Import der Bibliotheken.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.frequent_patterns import fpgrowth

## 9.1 Daten einlesen und bereinigen
Lesen Sie die Datei bank.csv ein und entfernen Sie die Splate "default".

In [2]:
df = pd.read_csv("09_Bankdaten.csv")
df.drop(["default"], axis=1, inplace=True)
df

,age,job,marital,education,balance,housing,loan,duration
0,58,management,married,tertiary,2143,yes,no,261
1,44,technician,single,secondary,29,yes,no,151
2,33,entrepreneur,married,secondary,2,yes,yes,76
3,47,blue-collar,married,unknown,1506,yes,no,92
4,33,unknown,single,unknown,1,no,no,198
...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,825,no,no,977
45207,71,retired,divorced,primary,1729,no,no,456
45208,72,retired,married,secondary,5715,no,no,1127
45209,57,blue-collar,married,secondary,668,no,no,508


In [3]:
df['duration'].describe()

count    45211.000000
mean       258.163080
std        257.527812
min          0.000000
25%        103.000000
50%        180.000000
75%        319.000000
max       4918.000000
Name: duration, dtype: float64

## 9.2 Binning der metrischen Attribute
Beide Algorithmen können nur mit binären Werten arbeiten, sodass zuerst kontinuierliche Werte diskretisiert werden müssen und anschließend alle Attribute in binäre Werte konvertiert werden müssen.

Teilen Sie dazu alle kontinuierlichen Attribute in 5-6 sinnvolle Bins ein und wenden dann One-Hot-Encoding an.

In [4]:
dfBins = df.copy()
bin_labels_age = ['unter 20', '20 bis 30', '30 bis 50', '50 bis 70', 'über 70']
dfBins['age'] = pd.cut(x = df['age'],
                              bins=[0, 20, 30, 50, 70, 100],
                              labels = bin_labels_age)

bin_labels_balance = ['negativ', 'niedrig', 'mittel', 'hoch', 'sehr hoch']
dfBins['balance'] = pd.cut(x = df['balance'],
                              bins=[-10000, 0, 75, 500, 1500, 110000],
                              labels = bin_labels_balance)

bin_labels_duration = ['sehr kurz', 'kurz', 'mittel', 'lang', 'sehr lang']
dfBins['duration'] = pd.cut(x = df['duration'],
                              bins=[0, 100, 300, 500, 700, 5000],
                              labels = bin_labels_duration)
dfBins

,age,job,marital,education,balance,housing,loan,duration
0,50 bis 70,management,married,tertiary,sehr hoch,yes,no,kurz
1,30 bis 50,technician,single,secondary,niedrig,yes,no,kurz
2,30 bis 50,entrepreneur,married,secondary,niedrig,yes,yes,sehr kurz
3,30 bis 50,blue-collar,married,unknown,sehr hoch,yes,no,sehr kurz
4,30 bis 50,unknown,single,unknown,niedrig,no,no,kurz
...,...,...,...,...,...,...,...,...
45206,50 bis 70,technician,married,tertiary,hoch,no,no,sehr lang
45207,über 70,retired,divorced,primary,sehr hoch,no,no,mittel
45208,über 70,retired,married,secondary,sehr hoch,no,no,sehr lang
45209,50 bis 70,blue-collar,married,secondary,hoch,no,no,lang


In [5]:
dfPrepared = pd.get_dummies(dfBins, prefix='',prefix_sep='')
dfPrepared.head()

,unter 20,20 bis 30,30 bis 50,50 bis 70,über 70,admin.,blue-collar,entrepreneur,housemaid,management,...,sehr hoch,no,yes,no,yes,sehr kurz,kurz,mittel,lang,sehr lang
0,0,0,0,1,0,0,0,0,0,1,...,1,0,1,1,0,0,1,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,1,1,0,0,1,0,0,0
2,0,0,1,0,0,0,0,1,0,0,...,0,0,1,0,1,1,0,0,0,0
3,0,0,1,0,0,0,1,0,0,0,...,1,0,1,1,0,1,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,1,0,1,0,0,1,0,0,0


## 9.3 Apriori & FPGrowth Methode

Finden Sie nun mit der Apriori und der FPGrowth Methode Assoziationsregeln mit min_support = 0.2 und Sortierungsmetrik confidence und lift.

#### A1: Apriori-Methode

In [6]:
dfApriori = apriori(dfPrepared, min_support=0.2, use_colnames=True)
dfApriori

,support,itemsets
0,0.639800,(30 bis 50)
1,0.215257,(blue-collar)
2,0.209197,(management)
3,0.601933,(married)
4,0.282896,(single)
5,0.513194,(secondary)
6,0.294198,(tertiary)
7,0.270576,(mittel)
8,0.235584,(hoch)
9,0.240981,(sehr hoch)


#### A2: FPGrowth-Methode

In [7]:
dfFpgrowth = fpgrowth(dfPrepared, min_support=0.2, use_colnames=True)
dfFpgrowth

,support,itemsets
0,0.839774,(no)
1,0.601933,(married)
2,0.555838,(yes)
3,0.487802,(kurz)
4,0.294198,(tertiary)
5,0.240981,(sehr hoch)
6,0.209197,(management)
7,0.639800,(30 bis 50)
8,0.513194,(secondary)
9,0.282896,(single)


#### 2.3 Interpretation der Ergebnisse

Welche Regeln erfüllen eine Konfidenz von > 0,8? Wie bewerten Sie diese Regeln inhaltlich?

In [9]:
dfApriori[dfApriori["support"] > 0.8]

,support,itemsets
12,0.839774,(no)


In [10]:
dfFpgrowth[dfFpgrowth["support"] > 0.8]

,support,itemsets
0,0.839774,(no)
